### Bias & Fairness in Data: Bias Mitigation Techniques
**Question**: Use the Adult Income dataset and apply reweighing technique to balance the
class weights based on sensitive attributes (e.g., gender).

In [3]:
import pandas as pd
import numpy as np
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing import Reweighing
import unittest

def load_adult_dataset():
    try:
        dataset = AdultDataset(
            protected_attribute_names=['sex'],
            privileged_classes=[['Male']],
            features_to_drop=['education-num']
        )
        return dataset
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

def apply_reweighing(dataset):
    try:
        train, _ = dataset.split([0.7], shuffle=True)
        rw = Reweighing(unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
        rw.fit(train)
        transformed_train = rw.transform(train)
        return train, transformed_train
    except Exception as e:
        print(f"Error during reweighing: {e}")
        return None, None

def analyze_weights(title, dataset):
    df = pd.DataFrame({
        'sex': dataset.features[:, dataset.feature_names.index('sex')],
        'label': dataset.labels.ravel(),
        'instance_weight': dataset.instance_weights
    })
    print(f"\n{title} - Mean instance weights:\n")
    print(df.groupby(['sex', 'label'])['instance_weight'].mean())

# --- Unit Test ---
class TestReweighing(unittest.TestCase):

    def test_reweighing_changes_weights(self):
        dataset = load_adult_dataset()
        train, transformed = apply_reweighing(dataset)

        # Ensure transformed weights are different
        self.assertIsNotNone(train)
        self.assertIsNotNone(transformed)
        original_weights = train.instance_weights
        new_weights = transformed.instance_weights

        self.assertFalse(np.allclose(original_weights, new_weights),
                         msg="Reweighing did not change the instance weights.")

if __name__ == "__main__":
    data = load_adult_dataset()
    if data:
        train_raw, train_rw = apply_reweighing(data)
        if train_rw:
            analyze_weights("Before Reweighing", train_raw)
            analyze_weights("After Reweighing", train_rw)

    unittest.main(argv=['first-arg-is-ignored'], exit=False)


ModuleNotFoundError: No module named 'aif360'